# Figure 8: Future regional differences

In [ ]:
import os 
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import shapely.geometry

os.chdir('/home/rooda/OneDrive/Projects/DeepHydro/')
path_geo = "/home/rooda/Datasets/GIS/"

## Data

In [ ]:
q_historical = xr.open_mfdataset("results/zenodo/Q_historical.nc")
q_historical = q_historical.resample(date = "YS").sum().mean(dim = "date")

In [ ]:
q_future = xr.open_mfdataset("results/zenodo/Q_future.nc").sel(date = slice("2070-01-01", "2099-12-31"))
q_future = q_future.resample(date = "YS").sum().mean(dim = "date")
q_future = (q_future / q_historical) - 1
q_future = q_future.load()

In [ ]:
# relative change for ssp126 and ssp585
q_future_lstm_oggm_mean = q_future.sel(model = "LSTM_OGGM_on").mean(dim = "gcm")
q_future_lstm_oggm_mean = q_future_lstm_oggm_mean.to_dataframe().reset_index()
q_future_lstm_oggm_mean = q_future_lstm_oggm_mean.pivot(index="basin_id", columns="ssp", values="Q")

q_future_lstm_oggm_sd = q_future.sel(model = "LSTM_OGGM_on").std(dim = "gcm")
q_future_lstm_oggm_sd = q_future_lstm_oggm_sd.to_dataframe().reset_index()
q_future_lstm_oggm_sd = q_future_lstm_oggm_sd.pivot(index="basin_id", columns="ssp", values="Q")
q_future_lstm_oggm_sd = q_future_lstm_oggm_sd.rename(columns = {"ssp126": "ssp126_sd", "ssp585": "ssp585_sd"})

In [ ]:
pmet_shape = gpd.read_file("data/GIS/Basins_Patagonia_all_data.gpkg") # Problem in Puelo 
pmet_shape = pmet_shape.set_index("gauge_id")
pmet_shape["total_area"] = pmet_shape.total_area*1e6 # in m2 
pmet_shape   = pd.concat([pmet_shape, q_future_lstm_oggm_mean, q_future_lstm_oggm_sd], axis = 1)

In [ ]:
# agreement all models on signal of change
q_future_agreement = q_future.mean(dim = "gcm")
q_future_agreement = q_future_agreement.where(q_future_agreement <= 0, -1).where(q_future_agreement> 0, 1)
q_future_agreement = q_future_agreement.sum(dim = "model")
q_future_agreement = q_future_agreement.to_dataframe().reset_index()
q_future_agreement = q_future_agreement.pivot(index="basin_id", columns="ssp", values="Q")

pmet_shape_subset126 = pmet_shape[(q_future_agreement.ssp126 == 4) | (q_future_agreement.ssp126 == -4)]
pmet_shape_subset585 = pmet_shape[(q_future_agreement.ssp585 == 4) | (q_future_agreement.ssp585 == -4)]

## Plot elements

In [ ]:
# basemap for background
geo_map = gpd.read_file(path_geo + "south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -54.99), (-76, -40.51), (-68.01, -40.51), (-68.01, -54.99), (-76, -54.99)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)

geo_map = geo_map.clip(poly_gdf)

In [ ]:
pmet_shape_main = pmet_shape[pmet_shape["total_area"] > 5000*1e6].copy()
pmet_shape_main["basin_name"] = ["Santa Cruz                          ", "Puelo", "Yelcho", "Grey","Palena", "Pascua", "Aysen", "Baker", "Cisnes"]

In [ ]:
from  plotly.colors import unlabel_rgb, hex_to_rgb
def binned_colorscale(seq, nr_swatches=5):
    
    if seq[0][0] == '#':
        arr_colors=np.array([hex_to_rgb(s) for s in seq])/255
    elif seq[0][0:3] == 'rgb':
        arr_colors = np.array([unlabel_rgb(s) for s in seq])/255 
    else:
        raise ValueError("a plotly colorscale is given either with hex colors or as rgb colors")
    n = len(seq)
    svals = [k/(n-1) for k in range(n)] #the scale values corresponding to the colors in seq
    grid = [k/(nr_swatches-1) for k in range(nr_swatches)]# define the scale values corresponding nr_swatches
    r, g, b = [np.interp(grid, svals, arr_colors[:, k]) for k in range(3)]  #np.interp interpolates linearly
    cmap_arr = np.clip(np.vstack((r, g, b)).T, 0, 1)
    new_colors = np.array(cmap_arr*255, dtype=int)
    discrete_colorscale = []
    N = len(new_colors+1)
    for k in range(N):
        discrete_colorscale.extend([[k/N, f'rgb{tuple(new_colors[k])}'], 
                                    [(k+1)/N,  f'rgb{tuple(new_colors[k])}']]) 
    return discrete_colorscale 

## Plot

In [ ]:
fig = make_subplots(rows=1, cols=2, vertical_spacing = 0.01, horizontal_spacing = 0.01,
                    specs=[[{"type": "scattergeo"}, {"type": "scattergeo"}]])

# colors
colorsc = binned_colorscale(["#fe7e0d", "#ffe9ba", "#1d78b4"], nr_swatches=15) 

dtick = 2
x = list(range(-74, 0 + dtick, dtick))
y = list(range(-56, 0 + dtick, dtick))
xpos = -75.9
ypos = -55.9

## Basemap -----------------------------------------------------------------------------------------------
for x_plot in range(0,2):
    fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                                colorscale = ["#EAEAF2", "#EAEAF2"],  showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=x_plot+1)

    fig.add_trace(go.Scattergeo(lon = [lon + 0.25 for lon in x[1:-1]] + [xpos + 0.25] * (len(y) - 2),
                                lat = [ypos + 0.1] * (len(x) - 2) + [lat + 0.1 for lat in y[1:-1]],
                                showlegend = False,
                                text = x[1:-1] + y[1:-1], textfont=dict(size=10, color = "rgba(0,0,0,0.25)"),
                                mode = "text"), row=1, col=x_plot+1)


fig.add_scattergeo(geojson = eval(pmet_shape_main['geometry'].to_json()), locations = pmet_shape_main.index, text = pmet_shape_main['basin_name'], mode = 'text', 
                       showlegend = False, textfont=dict(size=9, color = "rgba(0,0,0,0.25)"),row=1, col=1)


# relative differences
fig.add_trace(go.Choropleth(geojson = eval(pmet_shape['geometry'].to_json()),  locations = pmet_shape.index, z = pmet_shape['ssp126']*100, 
                            colorscale = colorsc, marker_line_color ='white', marker_line_width=0.2, 
                            zmin = -30, zmax = 30, colorbar=dict(len=0.95, x = 1.01, title='Projected<br>runoff<br>change<br>(%)', thickness=15, tickwidth=1)), row=1, col=1)

fig.add_trace(go.Choropleth(geojson = eval(pmet_shape['geometry'].to_json()),  locations = pmet_shape.index, z = pmet_shape['ssp585']*100, 
                            colorscale = colorsc, marker_line_color ='white', marker_line_width=0.2, 
                            zmin = -30, zmax = 30, showscale=False), row=1, col=2)

# agreement on the signal
fig.add_trace(go.Choropleth(geojson = eval(pmet_shape_subset126['geometry'].to_json()), locations = pmet_shape_subset126.index,  z = pmet_shape_subset126.ssp126, 
                            colorscale = ["rgba(0,0,0,0)", "rgba(0,0,0,0)"], marker_line_color ='black', marker_opacity = 0.8,
                            showscale= False, marker_line_width=0.2), row=1, col=1)

fig.add_trace(go.Choropleth(geojson = eval(pmet_shape_subset585['geometry'].to_json()), locations = pmet_shape_subset585.index,  z = pmet_shape_subset585.ssp585, 
                            colorscale = ["rgba(0,0,0,0)", "rgba(0,0,0,0)"], marker_line_color ='black', marker_opacity = 0.8,
                            showscale= False, marker_line_width=0.2), row=1, col=2)

## Annotations -----------------------------------------------------------------------------------------------
fig.add_annotation(text="a) SSP 1-2.6", x=0.01, y=0.01,  xref = "paper", yref = "paper", showarrow=False)
fig.add_annotation(text="b) SSP 5-8.5", x=0.58, y=0.01,  xref = "paper", yref = "paper", showarrow=False)

## Layout -------------------------------------------------------------------------------------------------------------------
fig.update_xaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", griddash = "dot", mirror=True)
fig.update_yaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", griddash = "dot", mirror=True)

fig.update_geos(showframe = True, framewidth = 1, framecolor = "rgba(0,0,0,0.5)", 
                lonaxis_range=[-76, -68], lataxis_range=[-56, -40.5], 
                bgcolor = "rgb(255,255,255)", 
                showland = False, showcoastlines = False, showlakes = False, 
                lataxis_showgrid=True, lonaxis_showgrid=True, 
                lonaxis_dtick = dtick, lataxis_dtick = dtick, 
                lonaxis_gridcolor = "rgba(0,0,0,0.1)", lataxis_gridcolor = "rgba(0,0,0,0.1)", 
                lonaxis_griddash = "dot", lataxis_griddash = "dot")

fig.update_layout(autosize = False, width = 700, height = 600, template = "seaborn", margin = dict(l=5, r=5, b=5, t=5))
fig.write_image("reports/figures/Figure8_regional_changes.png", scale = 4)
#fig.show()

## Text

In [ ]:
q_historical = q_historical.sel(model = "LSTM_OGGM_on").Q*pmet_shape.total_area/(365*1e3*86400)
pmet_shape["q_historical_flux"] = q_historical.rename("Q_historical").to_series()

In [ ]:
"minor relative changes in discharge ranging from {:.1f} ± {:.1f}% under SSP1-2.6 to {:.1f}% ± {:.1f}% under SSP5-8.5".format(
    (pmet_shape.q_historical_flux * (pmet_shape.ssp126)).sum()*100 / pmet_shape.q_historical_flux.sum(),
    (pmet_shape.q_historical_flux * (pmet_shape.ssp126_sd)).sum()*100 / pmet_shape.q_historical_flux.sum(),
    (pmet_shape.q_historical_flux * (pmet_shape.ssp585)).sum()*100 / pmet_shape.q_historical_flux.sum(),
    (pmet_shape.q_historical_flux * (pmet_shape.ssp585_sd)).sum()*100 / pmet_shape.q_historical_flux.sum())

In [ ]:
"SSP5-8.5: northern {:.1f} ± {:.1f}% central {:.1f}% ± {:.1f}% and southern {:.1f}% ± {:.1f}%".format(
    (pmet_shape.q_historical_flux * (pmet_shape.ssp585))[pmet_shape.lat > -46].sum()*100 / pmet_shape.q_historical_flux[pmet_shape.lat > -46].sum(),
    (pmet_shape.q_historical_flux * (pmet_shape.ssp585_sd))[pmet_shape.lat > -46].sum()*100 / pmet_shape.q_historical_flux[pmet_shape.lat > -46].sum(),
    (pmet_shape.q_historical_flux * (pmet_shape.ssp585))[(pmet_shape.lat < -46) & (pmet_shape.lat > -51)].sum()*100 / pmet_shape.q_historical_flux[(pmet_shape.lat < -46) & (pmet_shape.lat > -51)].sum(),
    (pmet_shape.q_historical_flux * (pmet_shape.ssp585_sd))[(pmet_shape.lat < -46) & (pmet_shape.lat > -51)].sum()*100 / pmet_shape.q_historical_flux[(pmet_shape.lat < -46) & (pmet_shape.lat > -51)].sum(),
    (pmet_shape.q_historical_flux * (pmet_shape.ssp585))[pmet_shape.lat < -51].sum()*100 / pmet_shape.q_historical_flux[pmet_shape.lat < -51].sum(),
    (pmet_shape.q_historical_flux * (pmet_shape.ssp585_sd))[pmet_shape.lat < -51].sum()*100 / pmet_shape.q_historical_flux[pmet_shape.lat < -51].sum())

In [ ]:
# Agreement
"Agreement between 4 model on the direction of change under SSP126 and SSP585 are {:.1f}% and {:.1f}%".format(
    pmet_shape_subset126.total_area.sum()*100/pmet_shape.total_area.sum(),
    pmet_shape_subset585.total_area.sum()*100/pmet_shape.total_area.sum())